In [2]:
import gensim.downloader as api
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.decomposition import PCA

import numpy as np
import pickle
import json
import copy
import pandas as pd
from tqdm import tqdm
import scipy.stats as stats


from Vocab import Vocab
vocab = Vocab('data/vocab.csv', 3)
from get_predictions import get_predictions
from helpers import *

from ChainDataset import ChainDataset

chain_test_set = ChainDataset(
    data_dir='data/',
    segment_file='segments.json',
    chain_file='test_chains.json',
    vectors_file='vectors.json',
    split='test'
)

from SegmentDataset import SegmentDataset

segment_test_set = SegmentDataset(
    data_dir='data/',
    segment_file='segments.json',
    vectors_file='vectors.json',
    split='test'
)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Initialising vocab from file.


## POS tags distribution


In [3]:
from collections import Counter
import nltk
import pickle
from nltk import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')

import matplotlib
import matplotlib.pyplot as plt

with open('oov_dictionary.pickle', 'rb') as f:
    oov_dict = pickle.load(f)

top_10 = ['NOUN', 'DET', 'VERB', 'ADP', 'ADJ', '.', 'ADV', 'NUM', 'PRON', 'CONJ']
top_4 = ['NOUN', 'VERB', 'ADJ', 'ADV']

''' explanation:

ADJ	adjective	new, good, high, special, big, local
ADP	adposition	on, of, at, with, by, into, under
ADV	adverb	really, already, still, early, now
CONJ	conjunction	and, or, but, if, while, although
DET	determiner, article	the, a, some, most, every, no, which
NOUN	noun	year, home, costs, time, Africa
NUM	numeral	twenty-four, fourth, 1991, 14:24
PRT	particle	at, on, out, over per, that, up, with
PRON	pronoun	he, their, her, its, my, I, us
VERB	verb	is, say, told, given, playing, would
.	punctuation marks	. , ; !
X	other	ersatz, esprit, dunno, gr8, univeristy

'''

oov_dict



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


{'grey': 'gray',
 'ok.': 'OK',
 't-shirt': 'tshirt',
 'w/': 'with',
 'frige': 'fridge',
 'red/white': 'red and white',
 'doughnuts': 'donuts',
 'didnt': "didn't",
 'ok..': 'ok',
 'dr.': 'dr',
 'no..': 'no',
 'doughnut': 'donut',
 'yes..': 'yes',
 'yellow/orange': 'yellow and orange',
 'pic..': 'picture',
 'umberlla': 'umbrella',
 'cest': 'chest',
 'k..next': 'ok, next',
 'surboard': 'surfboard',
 'blue/white': 'blue and white',
 'man/woman': 'man or woman',
 'doesnt': "doesn't",
 'scheppa': 'schepps',
 'guy/girl': 'guy or girl',
 'blue/green': 'blue and green',
 'schepp': 'schepps',
 'blue/black': 'blue and black',
 'motarcycle': 'motorcycle',
 'red/black': 'red and black',
 'red/white/blue': 'red, white and blue',
 'colour': 'color',
 'close-up': 'closeup',
 'green/yellow': 'green and yellow',
 'red/pink': 'red and pink',
 'buddist': 'buddhist',
 'longsleeves': 'long sleeves',
 'ihave': 'i have',
 'yellow/green': 'yellow and green',
 'no-': 'no',
 'k.': 'ok',
 'surfobard': 'surfboard'

In [4]:
dataset_pred_no_hist, dataset_pred_hist_cp, conditions_inds, condition_seg_hist, dataframe = get_pred_datasets()



loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test
history dataset with test_segments.json vectors.json test_chains.json test
processing test

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting predicitons took 36.962406158447266 seconds

  0%|          | 0/6801 [00:00<?, ?it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history
History


100%|██████████| 6801/6801 [00:00<00:00, 44055.38it/s]


In [5]:
# dataset_pred_no_hist, dataset_pred_hist_cp, conditions_inds, condition_seg_hist

dataset_pred_no_hist

In [6]:
def pos_tag_distribution_per_group(group):
    segments = list(condition_seg_hist[group])
    for segment in tqdm(segments):
        if condition_seg_hist[group][segment] == {}:
                continue

        # Get sentences per segment
        first_seg, current_seg = convert_to_sentences(segment, condition_seg_hist[group])
        first_sentence_dict = sentence_to_pos_tags(first_seg)
        current_sentence_dict = sentence_to_pos_tags(current_seg)

        # Update dicts
        update_dict(first_sentence_dict, total_first_seg_dict)
        update_dict(current_sentence_dict, total_current_dict)

    return total_first_seg_dict, total_current_dict

def convert_to_sentences(segment, dataset):
          
    # Decode first and current sentence 
    first_seg = list(dataset[segment].values())[0]['first_seg']

    current_seg = list(dataset[segment].values())[0]['current_seg']
    dec_first_seg = vocab.decode(first_seg)
    dec_current_seg = vocab.decode(current_seg)
    
    return dec_first_seg, dec_current_seg

def update_dict(new_dict, total_dict):
    for key, value in new_dict.items():
        new_key = key
        new_value = value
#         print("new key", new_key)
#         print("new_value", new_value)
        for key, value in total_dict.items():
            if key == new_key:
                current_value = total_dict[key] 
                update_value = current_value + new_value[0]
                total_dict[key] = update_value
                
    return None

def dict_to_result(first_seg_dict, group, result, place): #   convert dict to list
    
    dict_list = []
    total = 0
    for key, value in first_seg_dict.items():
        temp = [value,key]
        total += value
        dict_list.append(temp)
    
    # Convert to percentages
    for tag in dict_list:
        tag[0] = tag[0] / total
    
    dict_list.sort(reverse = True)
    group = group + "_" +  place
    
    result[group] = dict_list
    
    return result


def sentence_to_pos_tags(sentence):
    
    lower_case_sentence = []
    correct_sentence = [] 
    pos_tags = []
    

    for word in sentence:
        lower_case_sentence.append(word.lower())
    

    for word in lower_case_sentence:

        if word not in oov_dict:
            correct_sentence.append(word)
        else:
            correct_sentence.append(oov_dict[word])

    # Tag the sentence
    try:
        pos_tags += [word[1] for word in pos_tag(correct_sentence, tagset='universal')]

    except:
        None

        
    pos_tag_dict = dict()       
    for key,value in dict(Counter(pos_tags)).items():
        pos_tag_dict[key] = [value]  

    return pos_tag_dict


In [7]:
# groups = ['hT_nhF', 'hT_nhT', 'hF_nhF','hF_nhT']
groups = ['hT_nhF', 'hF_nhF']
result = {}
for group in groups:
    
    total_first_seg_dict = {
        'NOUN': 0,
        'DET': 0,
        'VERB': 0,
        'ADP': 0,
        'ADJ': 0,
        '.': 0,
        'ADV': 0,
        'NUM': 0,
        'PRON': 0,
        'CONJ': 0
    }

    total_current_dict = {
        'NOUN': 0,
        'DET': 0,
        'VERB': 0,
        'ADP': 0,
        'ADJ': 0,
        '.': 0,
        'ADV': 0,
        'NUM': 0,
        'PRON': 0,
        'CONJ': 0  
    }
    
    total_first_seg_dict, total_current_dict = pos_tag_distribution_per_group(group)
    result = dict_to_result(total_first_seg_dict, group, result, "first")
    result = dict_to_result(total_current_dict, group, result, "current")

dif_hT_nhF = {}
dif_hT_nhT = {}
dif_hF_nhF = {}
dif_hF_nhT = {}

# print(result)
for key, value in result.items():    
    if key.startswith('hT_nhF'):
        print("key", key)
        print("value", value)
        print("----")
        
        # als de dict leeg is vul dan
        if bool(dif_hT_nhF) == False:
            for v in value:
                dif_hT_nhF[v[1]] = v[0]
            
            
        # als dict al gevuld is .. trek dan de bende er vanaf
        else:
            pass
        
    if key.startswith("hT_nhT"):
        None
    if key.startswith("hF_nhF"):
        None
    if key.startswith("hF_nhT"):
        None
print(dif_hT_nhF)

100%|██████████| 248/248 [00:00<00:00, 321.03it/s]

key hT_nhF_first
value [[0.32154088050314467, 'NOUN'], [0.15683962264150944, 'VERB'], [0.13561320754716982, 'DET'], [0.10573899371069183, 'ADP'], [0.09905660377358491, 'ADJ'], [0.07389937106918239, '.'], [0.03970125786163522, 'PRON'], [0.029874213836477988, 'ADV'], [0.02397798742138365, 'CONJ'], [0.013757861635220126, 'NUM']]
----
key hT_nhF_current
value [[0.33168724279835393, 'NOUN'], [0.14814814814814814, 'VERB'], [0.14650205761316873, 'ADJ'], [0.09547325102880659, 'ADP'], [0.09382716049382717, 'DET'], [0.07901234567901234, '.'], [0.03786008230452675, 'PRON'], [0.02633744855967078, 'ADV'], [0.02551440329218107, 'NUM'], [0.015637860082304528, 'CONJ']]
----
{'NOUN': 0.32154088050314467, 'VERB': 0.15683962264150944, 'DET': 0.13561320754716982, 'ADP': 0.10573899371069183, 'ADJ': 0.09905660377358491, '.': 0.07389937106918239, 'PRON': 0.03970125786163522, 'ADV': 0.029874213836477988, 'CONJ': 0.02397798742138365, 'NUM': 0.013757861635220126}


In [8]:
def plot_dict(plot_title, values):
    labels = []
    all_values = []
    
    for v in range(len(values)):
        # print("v", values[v])

        labels.append(values[v][1])
        all_values.append(values[v][0])

    fig1, ax1 = plt.subplots()
    ax1.pie(all_values, labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    fig1.suptitle(plot_title, fontsize=20)
    plt.show()



In [9]:
test_dict = {
    'NOUN': 0,
    'DET': 0,
    'VERB': 0,
    'ADP': 0,
    'ADJ': 0,
    '.': 0,
    'ADV': 0,
    'NUM': 0,
    'PRON': 0,
    'CONJ': 0
}

new_dict = {
    'NOUN': 5,
    'ADV': 20,
}

for key, value in new_dict.items():
    new_key = key
    new_value = value
    print("new key", new_key)
    print("new_value", new_value)
    for key, value in test_dict.items():
        if key == new_key:
            print("found", key)
            test_dict[key] += new_value
            
print(test_dict)

new key NOUN
new_value 5
found NOUN
new key ADV
new_value 20
found ADV
{'NOUN': 5, 'DET': 0, 'VERB': 0, 'ADP': 0, 'ADJ': 0, '.': 0, 'ADV': 20, 'NUM': 0, 'PRON': 0, 'CONJ': 0}


## Experiments with leaving out tags in segments :)

In [10]:
print("Testing the SegmentDataset class item getter...")
print(f"Dataset contains {len(segment_test_set)} segment samples")
sample_id = 2
sample = segment_test_set[sample_id]
print(f"Segment {sample_id}:")
print(f"Image set: {sample['image_set']}")
print(f"Target image index(es): {sample["targets"]}")
# print("Target image Features: {}".format([segment_test_set.image_features[sample["image_set"][int(target)]] for target in sample["targets"]]))
print(f"Encoded segment: {sample['segment']}")
print(f"Decoded segment dialogue: {vocab.decode(sample["segment"])}")
print(f"Segment length: {sample["length"]}")
print("\nDone.")

SyntaxError: invalid syntax (<ipython-input-10-e5a11eb71476>, line 7)

In [11]:
print("Testing the ChainDataset class initialization...")
print(f"Dataset contains {len(chain_test_set.chains)} cains.")

sample_id = 5
sample = chain_test_set.chains[sample_id]

print("Chain {}:".format(sample_id))
print("Source Game ID: {}".format(sample["game_id"]))
print("Target image index: {}".format(sample["target"]))
print("Chain length: {}".format(len(sample["segments"])))
print("Segment IDs: {}".format(sample["segments"]))
print("Segment lengths: ", sample["lengths"])
print("First segment encoding: {}".format(chain_test_set.segments[sample["segments"][0]]["segment"]))
print("First segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][0]]["segment"])))
print("Second segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][1]]["segment"])))
# print("third segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][2]]["segment"])))

print("Reference chain and segments' associated image sets:")
for segment in sample["segments"]:
    print(vocab.decode(chain_test_set.segments[segment]["segment"]))
    print(chain_test_set.segments[segment]["image_set"])
print("\nDone.")

print("Testing the ChainDataset class item getter...")


Testing the ChainDataset class initialization...
Dataset contains 2811 cains.
Chain 5:
Source Game ID: 3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO
Target image index: 522129
Chain length: 4
Segment IDs: [3, 10, 12, 15]
Segment lengths:  [23, 23, 21, 19]
First segment encoding: [5, 38, 191, 23, 127, 11, 6, 120, 23, 127, 15, 9, 314, 10, 4, 13, 5, 80, 14, 128, 68, 22, 10]
First segment decoded dialogue: ['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']
Second segment decoded dialogue: ['-A-', 'do', 'you', 'have', 'the', 'one', 'with', 'two', 'plates', 'and', 'the', 'salsa', 'in', 'the', 'middle', '?', '-B-', 'i', 'do', "n't", 'have', 'that', 'one']
Reference chain and segments' associated image sets:
['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']


In [12]:
def create_history_dict(chain_data):
    history_dict = {}

    # Loop through all segments
    for segment in tqdm(range(0, 6801)):
        
        # For each segment, find its place in the chain_data and add this to history dict
        for index, chain_data_row in enumerate(chain_data):
            chain_segments = chain_data_row['segments']
            
            if segment in chain_segments:

                # If segment already exits, add new index
                if segment in history_dict:
                    current_list = history_dict[segment]
                    current_list.append(index)
                    history_dict[segment] = current_list
                # If segment does not yet exist in dict, add it 
                else:
                    history_dict[segment] = [index]
                    
    return history_dict
    # history_dict

#     for key, value in history_dict.items():
#         print(key)
#         print(value)


    # geef de segment_id en het returnt de index waar het in staat

    # segment_id: [ index]
    # {0: [0],
    #  7: [0],
    #  8: [0, 4],

In [13]:
def update_history_dataset(segment_dataset, history_dict, chain_data):
    
    for segment_id, segment_data in enumerate(segment_dataset):
        segment_length = segment_data['length']
        
        # Find at which index in history file a certain segment is
        index = history_dict[segment_id]
    
        # print("seg id ", segment_id)
        # print("seg length", segment_length)
    
        # print("index", index)
    
        # Retrieve all indices in history file and change length
        for i in index:
            # print("CHAIN", chain_data[i])
            all_segments_for_that_index = chain_data[i]['segments']
            for ii, seg in enumerate(all_segments_for_that_index):
                # print("ii", ii)
                # print("seg", seg)
                if segment_id == seg:
                    # print("HIER MOET JE LENGTE VERVANGEN", chain_data[i]['lengths'][0])
                    chain_data[i]['lengths'][ii] = segment_length
                    # print("DAARNA", chain_data[i])
                    
    return chain_data


In [123]:
def import_dialogue_data():
    segment_test_set = copy.deepcopy(segment_test_set)   
    chain_data = copy.deepcopy(chain_test_set.chains)
    
    return segment_test_set, chain_data

In [118]:
def remove_tag(tag, segment_test_set, remove_perc=1.0):
    for segment_id, segment_data in enumerate(segment_test_set):
        
        pos_tags = []
        lower_case_sentence = []
        correct_sentence = []

        encoded_segment = vocab.decode(segment_data['segment'])

        for word in encoded_segment:
            if word == '-A-':
                lower_case_sentence.append(word)
                continue
            if word == '-B-':
                lower_case_sentence.append(word)
                continue
            lower_case_sentence.append(word.lower())

        for word in lower_case_sentence:

            if word not in oov_dict:
                correct_sentence.append(word)
            else:
                correct_sentence.append(oov_dict[word])

        try: 
            pos_tags += [word[1] for word in pos_tag(correct_sentence, tagset='universal')]

            # List with indices to remove
            remove_index = []
            for index, word in enumerate(pos_tags):
                if word == tag:
                    remove_index.append(index)

            # Only keep certain percentage of indices to remove 
            remove_index = np.random.choice(remove_index, int(remove_perc*len(remove_index)))
          
            for index in reversed(remove_index):
                del correct_sentence[index]

            encoded_sentence = vocab.encode(correct_sentence)

            segment_test_set[segment_id]['segment'] = encoded_sentence
            segment_test_set[segment_id]['length'] = len(encoded_sentence)

        except IndexError:
            continue 

    
    return segment_test_set



In [120]:
history_dict = create_history_dict(chain_test_set.chains)

# Remove tags from nouns
segment_test_set_cp, chain_test_set_cp = import_dialogue_data()
segment_test_set_noun = remove_tag("NOUN", segment_test_set_cp, 0.5)
chain_data_noun = update_history_dataset(segment_test_set_noun, history_dict, chain_test_set_cp)

# Remove tags from verbs
segment_test_set, chain_data = import_dialogue_data()
segment_test_set_verb = remove_tag("VERB", segment_test_set)
chain_data_verb = update_history_dataset(segment_test_set_verb, history_dict, chain_data)

# Remove tags from adjectives
segment_test_set, chain_data = import_dialogue_data()
segment_test_set_adj = remove_tag("ADJ", segment_test_set)
chain_data_adj = update_history_dataset(segment_test_set_adj, history_dict, chain_data)

# Remove tags from adverbs
segment_test_set, chain_data = import_dialogue_data()
segment_test_set_adv = remove_tag("ADV", segment_test_set)
chain_data_adv = update_history_dataset(segment_test_set_adv, history_dict, chain_data)

100%|██████████| 6801/6801 [00:03<00:00, 2136.85it/s]


In [ ]:
# write chain information to file
with open('data/test_noun_chains.json', 'w') as json_file:
    json.dump(chain_data_noun, json_file)   

with open('data/test_verb_chains.json', 'w') as json_file:
    json.dump(chain_data_verb, json_file)
    
with open('data/test_adj_chains.json', 'w') as json_file:
    json.dump(chain_data_adj, json_file)

with open('data/test_adv_chains.json', 'w') as json_file:
    json.dump(chain_data_adv, json_file)

In [122]:
# write segments to json file
segment_data_noun_test = [seg for seg in segment_test_set_noun]
segment_data_verb_test = [seg for seg in segment_test_set_verb]
segment_data_adj_test = [seg for seg in segment_test_set_adj]
segment_data_adv_test = [seg for seg in segment_test_set_adv]

with open('data/test_noun_segments.json', 'w') as json_file:
    json.dump(segment_data_noun_test, json_file)
       
with open('data/test_verb_segments.json', 'w') as json_file:
    json.dump(segment_data_verb_test, json_file)
        
with open('data/test_adj_segments.json', 'w') as json_file:
    json.dump(segment_data_adj_test, json_file)
      
with open('data/test_adv_segments.json', 'w') as json_file:
    json.dump(segment_data_adv_test, json_file)
